<a href="https://colab.research.google.com/github/robimalco/colab/blob/main/Titanic_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
!kaggle competitions download -c titanic

In [ ]:
!mkdir train
!unzip train.zip -d train

# START

In [491]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from sklearn.preprocessing import LabelEncoder

In [422]:
train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()

In [423]:
train_df['AgeCluster'] = pd.cut(train_df['Age'], bins=[0, 10, 20, 40, 60, 120], include_lowest=True, labels=[0, 1, 2, 3, 4])
train_df['FareCluster'] = pd.cut(train_df['Fare'], bins=[0, 51, 101, 2000], include_lowest=True, labels=[0, 1, 2])
categorical_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'AgeCluster', 'FareCluster']

In [425]:
for column in categorical_columns:
    train_df[column] = LabelEncoder().fit_transform(train_df[column])

for column in categorical_columns:
  train_df[column] = train_df[column].astype('category')

In [434]:
pclass = train_df['Pclass'].cat.codes.values
sex = train_df['Sex'].cat.codes.values
sibsp = train_df['SibSp'].cat.codes.values
parch = train_df['Parch'].cat.codes.values
embarked = train_df['Embarked'].cat.codes.values
ageCluster = train_df['AgeCluster'].cat.codes.values
fareCluster = train_df['FareCluster'].cat.codes.values
categorical_data = np.stack([pclass, sex, sibsp, parch, embarked, ageCluster, fareCluster], 1)

In [437]:
tensor_data = torch.tensor(categorical_data, dtype=torch.int64)
tensor_output = torch.tensor(train_df['Survived'].values).flatten()

In [439]:
categorical_columns_size = [len(train_df[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_columns_size]
total_records = len(train_df)
test_records = int(total_records * 0.2)

In [440]:
tensor_train_data = tensor_data[:total_records-test_records]
tensor_test_data = tensor_data[total_records-test_records:total_records]
tensor_train_output = tensor_output[:total_records-test_records]
tensor_test_output = tensor_output[total_records-test_records:total_records]

In [535]:
class Model(nn.Module):
  def __init__(self, embedding_sizes):
    super().__init__()
    self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
    n_emb = 14
    self.lin1 = nn.Linear(n_emb, 200)
    self.lin2 = nn.Linear(200, 70)
    self.lin3 = nn.Linear(70, 2)
    self.bn1 = nn.BatchNorm1d(14)
    self.bn2 = nn.BatchNorm1d(200)
    self.bn3 = nn.BatchNorm1d(70)
    self.emb_drop = nn.Dropout(0.6)
    self.drops = nn.Dropout(0.3)
  def forward(self, x_cat):
    x = [e(x_cat[:,i]) for i, e in enumerate(self.embeddings)]
    x = torch.cat(x, 1)
    x = self.emb_drop(x)
    # x2 = self.bn1(20)
    # x = torch.cat([x, x2], 1)
    # x = F.relu(self.lin1(x))
    # x = self.drops(x)
    # x = self.bn2(x)
    # x = F.relu(self.lin2(x))
    # x = self.drops(x)
    # x = self.bn3(x)
    # x = self.lin3(x)
    return x

In [536]:
model = Model(categorical_embedding_sizes)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [537]:
epochs = 300
aggregated_losses = []

for i in range(epochs):
    i += 1
    y_pred = model(tensor_train_data)
    single_loss = loss_function(y_pred, tensor_train_output)

    aggregated_losses.append(single_loss)
    if i%25 == 1:
        print("epoch: " + str(i) + "\tloss: " + str(single_loss.item()))

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print("epoch: " + str(i) + "\tloss: " + str(single_loss.item()))

epoch: 1	loss: 4.070611000061035
epoch: 26	loss: 4.1089606285095215
epoch: 51	loss: 4.051157474517822
epoch: 76	loss: 3.8725287914276123
epoch: 101	loss: 3.786830425262451
epoch: 126	loss: 3.936457395553589
epoch: 151	loss: 3.9472763538360596
epoch: 176	loss: 3.84934401512146
epoch: 201	loss: 3.507652759552002
epoch: 226	loss: 3.6363375186920166
epoch: 251	loss: 3.7095375061035156
epoch: 276	loss: 3.5879197120666504
epoch: 300	loss: 3.420562982559204


In [476]:
with torch.no_grad():
    y_val = model(tensor_test_data)
    loss = loss_function(y_val.squeeze(), tensor_test_output)
print("Loss: " + str(loss))

Loss: tensor(0.4979)


In [477]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_val_sklearn = np.argmax(y_val, axis=1)

print("\nconfusion_matrix")
print(confusion_matrix(tensor_test_output, y_val_sklearn))
print("\nclassification_report")
print(classification_report(tensor_test_output, y_val_sklearn))
print("\naccuracy_score")
print(accuracy_score(tensor_test_output, y_val_sklearn))


confusion_matrix
[[ 3  7]
 [ 5 21]]

classification_report
              precision    recall  f1-score   support

           0       0.38      0.30      0.33        10
           1       0.75      0.81      0.78        26

    accuracy                           0.67        36
   macro avg       0.56      0.55      0.56        36
weighted avg       0.65      0.67      0.65        36


accuracy_score
0.6666666666666666
